# **Prior force field for the deep learning CG Model of DNA**

This Jupyter notebook train the network from the full atomistic trajectories ....

**All this notebook needs to be modified to use TorchMD-NET**

In [ ]:
import sys
import os
workDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(workDir, os.pardir))
sys.path.append(os.path.join(parentDir, 'src'))

import numpy as np

import torch.nn as nn
import torch
from torch.utils.data import DataLoader, RandomSampler
from torch.optim.lr_scheduler import MultiStepLR

from cgnet.feature import (MoleculeDataset, GeometryStatistics,
                           GeometryFeature, ShiftedSoftplus,
                           CGBeadEmbedding, SchnetFeature,
                           FeatureCombiner, LinearLayer,
                           GaussianRBF)
from cgnet.network import (HarmonicLayer, CGnet, ForceLoss,
                           lipschitz_projection, dataset_loss, Simulation)
from cgnet.molecule import CGMolecule

import matplotlib.pyplot as plt
%matplotlib inline

# We specify the training/simulating device here.
device = torch.device('cuda')

# Training

In [ ]:
delta_forces = np.load('DNA_Salt10_deltaforces.npy')
print("delta Force: {}".format(delta_forces.shape))

DNA_data = MoleculeDataset(CG_coords, delta_forces, embeddings, device=device)
print("Dataset length: {}".format(len(DNA_data)))

In [ ]:
# Hyperparameters

n_layers = 5
n_nodes = 128
activation = nn.Tanh()
batch_size = 512
learning_rate = 3e-4
rate_decay = 0.3
lipschitz_strength = 4.0

# schnet-specific parameters
n_embeddings = 10
n_gaussians = 50
n_interaction_blocks = 5
cutoff = 20.0

num_epochs = 20

save_model = False
directory = '.' # to save model

n_beads = CG_coords.shape[1]

In [ ]:
loader = DataLoader(DNA_data, sampler=RandomSampler(DNA_data),
                         batch_size=batch_size)
for num, batch in enumerate(loader):
    coords, forces, embeddings = batch
    print("Coordinates size:", coords.size())
    print("Forces size:", forces.size())
    print("Embeddings size:", embeddings.size())
    print(num, len(loader))
    break

In [ ]:
embedding_layer = CGBeadEmbedding(n_embeddings = n_embeddings,
                                  embedding_dim = n_nodes)

rbf_layer = GaussianRBF(high_cutoff=cutoff, n_gaussians=n_gaussians)

schnet_feature = SchnetFeature(feature_size = n_nodes,
                               embedding_layer = embedding_layer,
                               rbf_layer=rbf_layer,
                               n_interaction_blocks = n_interaction_blocks,
                               calculate_geometry = True,
                               n_beads = n_beads,
                               neighbor_cutoff = None,
                               device = device)

In [ ]:
layers = LinearLayer(n_nodes,
                     n_nodes,
                     activation=activation)

for _ in range(n_layers - 1):
    layers += LinearLayer(n_nodes,
                          n_nodes,
                          activation=activation)

# The last layer produces a single value
layers += LinearLayer(n_nodes, 1, activation=None)

DNA_model = CGnet(layers, ForceLoss(),
                 feature=schnet_feature,
                 priors=None).to(device)
print(DNA_model)

In [ ]:
optimizer = torch.optim.Adam(DNA_model.parameters(),
                             lr=learning_rate)
scheduler = MultiStepLR(optimizer,milestones=[10,20,30,40,50],
                        gamma=rate_decay)
epochal_train_losses = []
epochal_test_losses  = []
verbose = True

# printout settings
batch_freq = 500
epoch_freq = 1

In [ ]:
for epoch in range(1, num_epochs+1):
    train_loss = 0.00
    test_loss = 0.00
    n = 0
    for num, batch in enumerate(loader):
        optimizer.zero_grad()
        coord, force, embedding_property = batch

        
        energy, pred_force = DNA_model.forward(coord,
                                embedding_property=embedding_property)
        batch_loss = DNA_model.criterion(pred_force, force)
        batch_loss.backward()
        optimizer.step()
        
        # perform L2 lipschitz check and projection
        lipschitz_projection(DNA_model, strength=lipschitz_strength)
        if verbose:
            if (num+1) % batch_freq == 0:
                print(
                    "Batch: {: <5} Train: {: <20} Test: {: <20}".format(
                        num+1, batch_loss, test_loss)
                )
        train_loss += batch_loss.detach().cpu()
        n += 1

    train_loss /= n
    if verbose:
        if epoch % epoch_freq == 0:
            print(
                "Epoch: {: <5} Train: {: <20} Test: {: <20}".format(
    epoch, train_loss, test_loss))
    epochal_train_losses.append(train_loss)
    scheduler.step()
    
if save_model:
    torch.save(DNA_model,"{}/DNA_cgschnet.pt".format(directory))

In [ ]:
fig = plt.figure()
plt.plot(np.arange(0,len(epochal_train_losses),1),
         epochal_train_losses, label='Training Loss')
plt.legend(loc='best')
plt.xlabel("Epochs")
plt.xticks(np.arange(1,5))
plt.ylabel("Loss")
plt.show()

In [ ]:
pot, force = DNA_model.forward(torch.tensor(CG_coords[-1,:,:], , requires_grad=True),  torch.tensor(embeddings))